In [1]:
!pip install sgp4

  Obtaining dependency information for sgp4 from https://files.pythonhosted.org/packages/dd/fc/27496962d238fc18b9a005035141373307267830e3cf2b3bb04de7dfbebe/sgp4-2.23-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.6/158.6 kB 11.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
%matplotlib inline

In [4]:
import numpy as np

import sympy as sp

import my_orbit_lib.epoch
from my_orbit_lib.epoch import Epoch
from datetime import datetime

from sgp4.api import Satrec, jday, SGP4_ERRORS

import urllib.request

# Problem 3 - SGP4 Propagator

## Tinkering with SGP4

First let's test the library and see how it works. I'll use the example they provide at the website: https://pypi.org/project/sgp4/

In [5]:
s = '1 25544U 98067A   19343.69339541  .00001764  00000-0  38792-4 0  9991'
t = '2 25544  51.6439 211.2001 0007417  17.6667  85.6398 15.50103472202482'
satellite = Satrec.twoline2rv(s, t)

And propagate it to, quote, 12:50:19 on 29 June 2000:

In [6]:
jd, fr = 2458827, 0.362605
e, r, v = satellite.sgp4(jd, fr)
e

0

In [7]:
r, v  # in True Equator Mean Equinox coordinates 

((-6102.443287145759, -986.3320567914377, -2820.3130331545203),
 (-1.4552526713308918, -5.5274138264242625, 5.101042055899729))

0 for e means there's no error. What's interesting is that they divide the JDE into two parts, the whole JD part and the fractional part for the hms. Since I wrote a JD function preliminary in problem 1, let's compare it to what this library expects. Without looking at the source code we'll see if the conversions match up. 

In [8]:
Epoch(2458827.362605).todatetime()

datetime.datetime(2019, 12, 9, 20, 42, 9, 71993)

.. now this is certainly not the year 2000. However I suspect the date they quote just before the example is wrong. For one, again, using the [US Naval Observatory's JD Converter](https://aa.usno.navy.mil/data/JulianDate) for 2458827.362605 we get 2019/12/09 at 20:42:9.1

Moreover, the library itself has a date to JD function which gives the same:

In [9]:
jd, fr = jday(2019, 12, 9, 20, 42, 9)
jd + fr

2458827.362604167

Even though I got scared at the beginning it seems like it's really a typo in their example.

Also, to get the same conversion, for our part we shouldn't pay attention to leap seconds, it seems like they are using TT by default. So let's disable it with the flag:

In [10]:
epoch = Epoch(datetime(2019, 12, 9, 20, 42, 9), utc=False)
epoch.jde

2458827.3626041664

In [11]:
def epoch_to_jd_fr(epoch):
    fr = epoch.jde % 0.5
    jd = epoch.jde - fr
    return jd, fr

epoch_to_jd_fr(epoch)

(2458827.0, 0.3626041663810611)

The documentation says the library accepts e.g. 2458827.5 for the whole part, which signifies half a day. So we mod it by 0.5 and not 1.

## Propagating the Position of AQUA

Now let's apply it to our satellite of interest:

In [12]:
# Note that TLE is a format which is really really column specific.
# e.g. pasting the lines below from the pdf blindly results in improper
# parsing and then the SGP4 lib silently consumes it, giving errors later 
# when propagating.

# The fix is to ensure the proper spacing and pay attention to where 
# stuff should sit according to https://celestrak.org/NORAD/documentation/tle-fmt.php

'0 AQUA'

# e.g. this is wrong, the columns are missing spaces:
s_old = '1 27424U 02022A 23031.44029486 .00000919 00000-0 21178-3 0 9992'
t_old = '2 27424 98.2732 336.4878 0000767 102.4680 35.7414 14.57638807103436'

# this is right:
s_old = '1 27424U 02022A   23031.44029486  .00000919  00000-0  21178-3 0  9992'
t_old = '2 27424  98.2732 336.4878 0000767 102.4680  35.7414 14.57638807103436'

aqua_old = Satrec.twoline2rv(s_old, t_old)

In [13]:
# 16:23:05 on the 1st of February 2023
aqua_tbd = Epoch(datetime(2023, 2, 1, 16, 23, 5), utc=False)
jd, fr = epoch_to_jd_fr(aqua_tbd)
e, r, v = aqua_old.sgp4(jd, fr)
if e != 0:
    print(SGP4_ERRORS[e])

In [14]:
r, v # in True Equator Mean Equinox coordinates

((-6556.911810573834, 2570.8682033473106, 733.2756194022593),
 (-0.3282354380158116, 1.2941755175511651, -7.385186412850507))

## Getting the Latest Info for AQUA

In [15]:
with urllib.request.urlopen('https://celestrak.org/NORAD/elements/active.txt') as response:
    lines = [line.decode().strip() for line in response]

In [16]:
for i, l in enumerate(lines):
    if l.startswith('AQUA'):
        break

In [17]:
i, l

(354, 'AQUA')

In [18]:
s_newest, t_newest = lines[i + 1], lines[i + 2]
s_newest, t_newest

('1 27424U 02022A   23338.10562127  .00001725  00000+0  38031-3 0  9990',
 '2 27424  98.3074 282.1332 0001660  55.2068  72.5345 14.58632910148123')

In [19]:
# 1 NNNNNU NNNNNAAA NNNNN.NNNNNNNN +.NNNNNNNN +NNNNN-N +NNNNN-N N NNNNN
#   ^ sat number
#
#          ^^^^^^^^ designators
#
#                   ^^   epoch year (columns 19-20)
#
#                     ^^   Epoch (Day of the year and fractional portion of the day) (columns 21-32) 

In [20]:
s_newest[18:20], s_newest[20:32]

('23', '338.10562127')

According to the format this means the 67th day of 2023 and the fractional part of the day.

In [21]:
year = int('20' + s_newest[18:20])
fr_day = float(s_newest[20:32])
year, fr_day

(2023, 338.10562127)

[TLE FQA](https://celestrak.org/columns/v04n03/#FAQ02) states that an epoch of 98000.00000000 would actually correspond to the beginning of 1997 December 31. So to convert properly to date time we must start at December 31 of the previous year:

In [22]:
aqua_latest_epoch = Epoch(Epoch(datetime(year - 1, 12, 31), utc=False).jde + fr_day)
aqua_latest_epoch

Epoch(2460282.60562127)

In [23]:
aqua_latest_epoch.todatetime()

datetime.datetime(2023, 12, 4, 2, 32, 5, 677720)

In [24]:
jd, fr = epoch_to_jd_fr(aqua_latest_epoch)
e, aqua_r_from_old_to_latest, aqua_v_from_old_to_latest = aqua_old.sgp4(jd, fr)
if e != 0:
    print(SGP4_ERRORS[e])

This gives us the propagated version from the old TLE:

In [25]:
aqua_r_from_old_to_latest, aqua_v_from_old_to_latest

((788.409987245059, 1195.5388445405724, -6937.563915150476),
 (1.6188594141752017, -7.2367076490400155, -1.0636455586543232))

To compare it with the position information from latest TLE at the latest TLE epoch:

In [26]:
aqua_latest = Satrec.twoline2rv(s_newest, t_newest)
jd, fr = epoch_to_jd_fr(aqua_latest_epoch)
e, aqua_r_latest, aqua_v_latest = satellite.sgp4(jd, fr)
if e != 0:
    print(SGP4_ERRORS[e])

In [27]:
aqua_r_latest, aqua_v_latest

((-5411.931353588816, 3493.678215831159, -2087.7444853912166),
 (-4.218557269257569, -3.234703903412092, 5.537025864594144))

It's 5 weeks into the future, so a pretty big error:

In [28]:
np.linalg.norm(np.array(aqua_r_from_old_to_latest) - np.array(aqua_r_latest))

8200.391808566217